# Predicting the flow 

## Decision tree regression

### Importing necessary libraries

In [17]:
import pandas as pd
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from sklearn.tree import DecisionTreeRegressor
import datetime as dt

### Loading the data set

In [18]:
flow = pd.read_csv('../dataset/flow1.csv', parse_dates=['time'])
flow.head()

,time,flow
0,2015-06-01 14:15:00,115.559998
1,2015-06-01 14:20:00,115.199997
2,2015-06-01 14:25:00,112.209999
3,2015-06-01 14:30:00,112.860001
4,2015-06-01 14:35:00,113.349998


### Preparing the data 
Making the mean for each hour, reseting the index put on the time and leaving only integer representing the current hour

In [43]:
flow_h = pd.read_csv('../dataset/flow1.csv', parse_dates={'hour':['time']},index_col='hour')
dayfirst=True
flow_h = flow_h.resample('60T').mean()

In [44]:
flow_h = flow_h.reset_index()

In [45]:
flow_h['time'] = flow_h['hour']
flow_h.head()

,hour,flow,time
0,2015-06-01 14:00:00,113.754443,2015-06-01 14:00:00
1,2015-06-01 15:00:00,113.013333,2015-06-01 15:00:00
2,2015-06-01 16:00:00,114.081666,2015-06-01 16:00:00
3,2015-06-01 17:00:00,116.459167,2015-06-01 17:00:00
4,2015-06-01 18:00:00,123.825000,2015-06-01 18:00:00


In [46]:
flow_h['hour'] = (flow_h['hour'].dt.hour)

In [47]:
flow_h.head()

,hour,flow,time
0,14,113.754443,2015-06-01 14:00:00
1,15,113.013333,2015-06-01 15:00:00
2,16,114.081666,2015-06-01 16:00:00
3,17,116.459167,2015-06-01 17:00:00
4,18,123.825000,2015-06-01 18:00:00


### Splitting the data set into train and test sets

In [31]:
split_date = pd.datetime(2016,12,31,23,59)

flow_training = flow_h.loc[flow_h['time'] <= split_date]
flow_test = flow_h.loc[flow_h['time'] > split_date]

X_flow_training = flow_training['hour']
Y_flow_training = flow_training['flow']

X_flow_test = flow_test['hour']
Y_flow_test = flow_test['flow']

In [48]:
flow_training.head()

,hour,flow,time
0,14,113.754443,2015-06-01 14:00:00
1,15,113.013333,2015-06-01 15:00:00
2,16,114.081666,2015-06-01 16:00:00
3,17,116.459167,2015-06-01 17:00:00
4,18,123.825000,2015-06-01 18:00:00


In [49]:
flow_test.head()

,hour,flow,time
13906,0,94.330833,2017-01-01 00:00:00
13907,1,91.927501,2017-01-01 01:00:00
13908,2,90.722501,2017-01-01 02:00:00
13909,3,87.402501,2017-01-01 03:00:00
13910,4,81.485833,2017-01-01 04:00:00


### Creating a model

### Implementing metrics (mean absolute percentage error)

In [1]:
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100